#### Transfer entropy
Transfer entropy is commonly used with time series data because it's a measure of the influence of one variable on another over time. It helps determine how much historical data points of one variable influence the future values of another variable. This is particularly useful for analyzing cause-and-effect relationships, information flow, or dependencies in time-ordered data.

In [1]:
# import libs
import numpy as np
from scipy.stats import entropy
import pandas as pd

In [2]:
# code for transfer entropy
def transfer_entropy(data, features, target):
    te_scores = {}
    
    for feature in features:
        for lag in range(1, len(data)):
            x_t = data[feature].shift(-lag).dropna().values
            x_t_past = data[feature].shift(lag).dropna().values
            y_t = data[target][lag:].values
            
            h_x_t_given_x_t_past = entropy(x_t, x_t_past)
            h_x_t_given_x_t_past_y_t = entropy(x_t, [x_t_past, y_t])
            
            te = h_x_t_given_x_t_past - h_x_t_given_x_t_past_y_t
            
            if feature not in te_scores:
                te_scores[feature] = []
            te_scores[feature].append((lag, te))
    
    return te_scores

In [3]:
data = pd.DataFrame({
    'Temperature': [25, 26, 27, 28, 29],
    'Humidity': [60, 65, 70, 75, 80],
    'IceCreamSales': [100, 105, 110, 115, 120]
})

data.head()

,Temperature,Humidity,IceCreamSales
0,25,60,100
1,26,65,105
2,27,70,110
3,28,75,115
4,29,80,120


In [4]:
features = ['Temperature','Humidity']
target = 'IceCreamSales'
te_scores = transfer_entropy(data, features, target)

In [5]:
for feature, scores in te_scores.items():
    print(f"Transfer Entropy from {feature} to {target}:")
    for lag, te in scores:
        print(f"  Lag {lag}: {te}")

Transfer Entropy from Temperature to IceCreamSales:
  Lag 1: [0.24169125 0.23135538 0.22035268 0.20870648]
  Lag 2: [0.12174889 0.10091367 0.07923652]
  Lag 3: [-0.24452304 -0.29005591]
  Lag 4: [-1.94709992]
Transfer Entropy from Humidity to IceCreamSales:
  Lag 1: [0.34236505 0.33506716 0.32523235 0.31303423]
  Lag 2: [0.26710497 0.24357868 0.21695486]
  Lag 3: [-0.01856092 -0.08050453]
  Lag 4: [-1.5040774]


The results you've obtained represent the Transfer Entropy values for different time lags when trying to predict "IceCreamSales" from "Temperature." Each "Lag" corresponds to a different time shift between the two variables. Let's interpret the results:

1. **Lag 1:**
   - The Transfer Entropy values are positive, indicating that the past "Temperature" values (lag 1) help predict future "IceCreamSales." The higher the value, the more information is transferred from "Temperature" to "IceCreamSales."
   - It suggests that there's a moderate influence of yesterday's temperature on today's ice cream sales.
   - The values decrease as the lag increases, which is expected because the influence of past temperature decreases as it gets further from the current day.

2. **Lag 2:**
   - Transfer Entropy values are still positive but lower than at lag 1. This means that the temperature from two days ago has a smaller influence on today's ice cream sales.
   - It suggests a decreasing but still moderate influence from the past.

3. **Lag 3:**
   - Transfer Entropy values are negative. This means that knowing the temperature three days ago doesn't help predict ice cream sales today. In fact, it may even lead to reduced predictability. This is because at lag 3, you're looking too far into the past for relevant information.

4. **Lag 4:**
   - Transfer Entropy is highly negative. This suggests that there's almost no transfer of information between temperature four days ago and ice cream sales today. At this lag, the information from temperature is almost unhelpful in predicting ice cream sales.

In summary, these results show that there is a time-dependent relationship between "Temperature" and "IceCreamSales." Temperature from yesterday has a moderate positive influence, but as you look further into the past, this influence decreases and can even become negative, indicating a lack of influence. These findings can be valuable for understanding how temperature affects ice cream sales and at what time scale this influence is most significant.